In [1]:
import pandas as pd

import sys
import pathlib

import statsmodels.api as sm
#import pingouin as pg
import plotly.express as px

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, HBox, Dropdown, IntSlider, interact
from IPython.display import display

%matplotlib inline

In [2]:
import logging
import os


os.makedirs("../../logs", exist_ok=True)
# Ensure the file is created if it doesn't exist
log_filename = "../../logs/my_log_file.log"
if not os.path.exists(log_filename):
    open(log_filename, "w").close()  # This just creates an empty file

logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    filemode="w",
    format="%(asctime)s - %(levelname)s - %(message)s",
)

In [3]:
#sys.path.append("G:/Shared drives/UGS_Flux/Data_Processing/Jupyter_Notebooks/MicroMet")
#sys.path.append("C:/Users/kladig/Documents/GitHub/MicroMet")
sys.path.append("../..")
import micromet

In [4]:
# dictionary of station ids as keys and values as subdirectories
site_folders = {#'US-UTD':'Dugout_Ranch',
                #'US-UTB':'BSF',
                #'US-UTJ':'Bluff',
                #'US-UTW':'Wellington',
                #'US-UTE':'Escalante',
                'US-UTM':'Matheson',
                #'US-UTP':'Phrag',
                #'US-CdM':'Cedar_Mesa',
                #'US-UTV':'Desert_View_Myton',
                }

compdf = {}
am = micromet.AmerifluxDataProcessor()

for key, value in site_folders.items():

    print(key)
    raw_fold = pathlib.Path('G:/Shared drives/UGS_Flux/Data_Downloads/')
    amflux_out = 'G:/Shared drives/UGS_Flux/Data_Processing/Ameriflux/'
    raw_data = am.raw_file_compile(raw_fold, value, search_str = "*Flux_AmeriFluxFormat*.dat")
    if raw_data is not None:
        am_data = micromet.Reformatter(raw_data)
        am_df = am_data.et_data
        compdf[key] = am_df
        micromet.outfile(am_df, key, amflux_out)

        
cdf = pd.concat(compdf,axis=0)

# make lists for selection dropdowns in the interactive parts of the code
categories = cdf.index.get_level_values(0).unique()
fields = list(cdf.columns)

US-UTM
Unknown header format for file: G:\Shared drives\UGS_Flux\Data_Downloads\Matheson\8442_Flux_AmeriFluxFormat_0.dat
Unknown header format for file: G:\Shared drives\UGS_Flux\Data_Downloads\Matheson\d20211228\8442_Flux_AmeriFluxFormat_0.dat
Unknown header format for file: G:\Shared drives\UGS_Flux\Data_Downloads\Matheson\d20220215\8442_Flux_AmeriFluxFormat_0.dat
Unknown header format for file: G:\Shared drives\UGS_Flux\Data_Downloads\Matheson\d20220323\8442_Flux_AmeriFluxFormat_0.dat


In [4]:

def filter_data(index_value):
    #index_value = change['new']

    # Check if input index exists in DataFrame
    if index_value in cdf.index:
        display(cdf.loc[index_value])
    else:
        print(f"Index {index_value} does not exist in DataFrame.")

index_input = interact(filter_data,
    index_value = widgets.Dropdown(
        options=categories,
        value=categories[0],
        description='Categories',
        disabled=False
    ))

interactive(children=(Dropdown(description='Categories', options=('US-UTM',), value='US-UTM'), Output()), _dom…

# Interactive intital Plots

In [4]:
# Create interactive widgets with date range
interact(
    micromet.plot_timeseries_daterange,
    input_df = widgets.fixed(cdf),
    selected_station=widgets.Dropdown(
        options=categories,
        value=categories[0],
        description='Categories',
        disabled=False
    ),
    selected_field=widgets.Dropdown(
        options=fields,
        value=fields[0],
        description='Field',
        disabled=False
    ),
    start_date=widgets.DatePicker(
        description='Start Date',
        value=pd.to_datetime('2020-01-01')
    ),
    end_date=widgets.DatePicker(
        description='End Date',
        value=pd.to_datetime('2024-12-31')
    )
)

save_button = widgets.Button(description="Save Plot")
save_button.on_click(micromet.save_plot)
display(save_button)


NameError: name 'cdf' is not defined